# Analyze NBI bridges for SPC planning region

In [1]:
import pandas as pd

### Import 2022 NBI data for Pennsylvania

In [2]:
nbi = pd.read_csv('input/nbi22.csv')
nbi['STRUCTURE_NUMBER_008'] = nbi.STRUCTURE_NUMBER_008.astype(str).str.zfill(15)

/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_81450/1129763336.py:1: DtypeWarning: Columns (1,17,75) have mixed types. Specify dtype option on import or set low_memory=False.
  nbi = pd.read_csv('input/nbi22.csv')


In [3]:
len(nbi)

23202

In [4]:
nbi.head(1)

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,BRIDGE_LEN_IND_112,SCOUR_CRITICAL_113,FUTURE_ADT_114,YEAR_OF_FUTURE_ADT_115,MIN_NAV_CLR_MT_116,FED_AGENCY,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA
0,42,1PA0099,1,8,0,00000,0,0,41,42635,...,Y,5,1800,2035,NaN,Y,73,F,6,88.75


### Limit to SPC planning region

In [5]:
nbi = nbi[nbi.COUNTY_CODE_003.isin([3, 5, 7, 19, 51, 59, 63, 73, 125, 129])]

In [6]:
len(nbi)

4788

### Introduce custom condition field

In [7]:
nbi['BridgeConditionNum'] = nbi[[
    'DECK_COND_058',
    'SUPERSTRUCTURE_COND_059',
    'SUBSTRUCTURE_COND_060',
    'CULVERT_COND_062'
]].min(axis=1)

nbi.loc[nbi.BridgeConditionNum >= 7, 'BridgeCondition'] = 'G'
nbi.loc[nbi.BridgeConditionNum.isin([5, 6]), 'BridgeCondition'] = 'F'
nbi.loc[nbi.BridgeConditionNum <= 4, 'BridgeCondition'] = 'P'

### Introduce custom owner field

In [8]:
nbi['SimpleOwner'] = nbi.OWNER_022

In [9]:
nbi['SimpleOwner'] = nbi.SimpleOwner.replace({
    1: 'State',
    2: 'County',
    3: 'Municipal',
    4: 'Municipal',
    11: 'State',
    21: 'Other',
    25: 'Other',
    26: 'Other',
    27: 'Other',
    31: 'State',
    32: 'Other',
    66: 'Federal',
    70: 'Federal'
    
})

### Look at all bridges by owner, condition

In [10]:
piv = pd.pivot_table(nbi[['SimpleOwner', 'BridgeCondition']], index=['SimpleOwner'], columns='BridgeCondition', values='BridgeCondition', aggfunc='size')
piv['Total'] = piv.sum(axis=1)

In [11]:
piv['P%'] = 100 * (piv.P / piv.Total)
piv['P%_total'] = 100 * (piv.P / piv.P.sum())

In [12]:
piv

BridgeCondition,F,G,P,Total,P%,P%_total
SimpleOwner,,,,,,
County,341,199,133,673,19.762259,21.077655
Federal,1,1,1,3,33.333333,0.158479
Municipal,328,139,201,668,30.089820,31.854200
Other,33,6,21,60,35.000000,3.328051
State,1793,1316,275,3384,8.126478,43.581616


### Look at off-system bridges by owner, condition

In [13]:
piv = pd.pivot_table(nbi[nbi.FUNCTIONAL_CLASS_026.isin([8, 9, 19])][['SimpleOwner', 'BridgeCondition']], index=['SimpleOwner'], columns='BridgeCondition', values='BridgeCondition', aggfunc='size')
piv['Total'] = piv.sum(axis=1)

In [14]:
piv['P%'] = 100 * (piv.P / piv.Total)
piv['P%_total'] = 100 * (piv.P / piv.P.sum())

In [15]:
piv

BridgeCondition,F,G,P,Total,P%,P%_total
SimpleOwner,,,,,,
County,247,148,94,489,19.222904,22.705314
Federal,1,1,1,3,33.333333,0.241546
Municipal,303,127,180,610,29.508197,43.478261
Other,12,4,12,28,42.857143,2.898551
State,373,317,127,817,15.544676,30.676329


# Analyze formerly poor NBI bridges for SPC planning region

In [16]:
formerlypoor = pd.concat([
    pd.read_csv('input/PA_Poor12_to_Fair22.csv', quotechar="'"),
    pd.read_csv('input/PA_Poor12_to_Good22.csv', quotechar="'"),
])

In [17]:
formerlypoor['8 - Structure Number'] = formerlypoor['8 - Structure Number'].astype(str).str.zfill(15)

In [18]:
formerlypoor = pd.merge(
    formerlypoor[['8 - Structure Number']],
    nbi,
    left_on='8 - Structure Number',
    right_on='STRUCTURE_NUMBER_008'
)

formerlypoor['SimpleOwner'] = formerlypoor.OWNER_022
formerlypoor['SimpleOwner'] = formerlypoor.SimpleOwner.replace({
    1: 'State',
    2: 'County',
    3: 'Municipal',
    4: 'Municipal',
    11: 'State',
    21: 'Other',
    25: 'Other',
    26: 'Other',
    27: 'Other',
    31: 'State',
    32: 'Other',
    66: 'Federal',
    70: 'Federal'
})

In [19]:
formerlypoor.SimpleOwner.value_counts()

State        235
County        79
Municipal     35
Other          5
Name: SimpleOwner, dtype: int64